In [1]:
import os
import sys
HOME = os.path.abspath('..')
sys.path.append(HOME)
os.chdir(HOME)
import pandas as pd
#!pip install transformers
from transformers import RobertaConfig, RobertaModel,RobertaForSequenceClassification, Trainer
from transformers import AutoTokenizer
from transformers.models.roberta import RobertaPreTrainedModel
import torch
from torch import nn
from transformers import TrainingArguments

In [58]:
a=1

list(range(1, a+1, a//1))

[1]

In [3]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def process_text_data(data_:pd.DataFrame,text_col,na_filler = tokenizer.unk_token):
    data = data_.copy()
    data[text_col] = data[text_col].fillna(na_filler)
    encodings = tokenizer(data[text_col].tolist(), padding=False, truncation=True)
    return encodings
    
    

In [44]:
all_cols =  ['Budget',
             'averageRating',
             'cast',
             'countries',
             'director',
             'genres',
             'imdb_id',
             'languages',
             'overview',
             'production companies',
             'release_date',
             'revenue_worldwide_BOM',
             'runtimeMinutes',
             'title']


train_ids = pd.read_csv('data/processed/train.csv',usecols=['imdb_id'])['imdb_id'].tolist()
val_ids = pd.read_csv('data/processed/val.csv',usecols=['imdb_id'])['imdb_id'].tolist()
test_ids = pd.read_csv('data/processed/test.csv',usecols=['imdb_id'])['imdb_id'].tolist()
df = pd.read_csv('data/processed/df.csv',usecols = all_cols)

TARGET_COL = 'averageRating'

In [5]:
train = df[df['imdb_id'].isin(train_ids)]
val = df[df['imdb_id'].isin(val_ids)]
test = df[df['imdb_id'].isin(test_ids)]

In [6]:
assert len(set(train['imdb_id']) & set(val['imdb_id'])) == 0
assert len(set(train['imdb_id']) & set(test['imdb_id'])) == 0
assert len(set(test['imdb_id']) & set(val['imdb_id'])) == 0

In [36]:
max(len(i) for i in train_encodings['input_ids'])

296

In [38]:
train.columns

Index(['imdb_id', 'runtimeMinutes', 'genres', 'cast', 'averageRating',
       'numVotes', 'title', 'original_title', 'overview', 'release_date',
       'poster_link', 'revenue_worldwide_BOM', 'director', 'countries',
       'country codes', 'language codes', 'languages', 'Budget', 'cover url',
       'production companies'],
      dtype='object')

In [48]:
train['genres'][0].replace('|',', ')

'comedy, fantasy, romance'

In [50]:
train['cast'][0].replace('|',', ')

'meg ryan, hugh jackman, liev schreiber'

In [49]:
train['genres'][0]

'comedy|fantasy|romance'

In [42]:
df[['Year', 'quarter', ...]].agg('-'.join, axis=1)

In [7]:
#Get train encodings
train_encodings = process_text_data(train,'overview')
val_encodings = process_text_data(val,'overview')
test_encodings = process_text_data(test,'overview')

#get labels
train_labels = train[TARGET_COL].tolist()
val_labels = val[TARGET_COL].tolist()
test_labels = test[TARGET_COL].tolist()

#Create dataset objects
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [32]:
train

,imdb_id,runtimeMinutes,genres,cast,averageRating,numVotes,title,original_title,overview,release_date,poster_link,revenue_worldwide_BOM,director,countries,country codes,language codes,languages,Budget,cover url,production companies
0,35423.0,118,comedy|fantasy|romance,meg ryan|hugh jackman|liev schreiber,6.4,83205.0,Kate & Leopold,Kate & Leopold,When her scientist ex-boyfriend discovers a po...,2001-12-25,https://images-na.ssl-images-amazon.com/images...,76019048.0,james mangold,united states,us,en|fr,english|french,48000000.0,https://m.media-amazon.com/images/M/MV5BNmNlN2...,konrad pictures|miramax
1,79285.0,88,adventure|horror|sci-fi,farrah fawcett|kirk douglas|harvey keitel,5.1,9337.0,Saturn 3,Saturn 3,"In the future, Earth is overcrowded and the po...",1980-02-15,NaN,9000000.0,stanley donen|john barry,united kingdom,gb,en,english,10000000.0,https://m.media-amazon.com/images/M/MV5BOGU1MD...,itc films|elliott kastner productions|transcon...
2,79302.0,90,comedy,nancy allen|mary davenport|kirk douglas,5.1,911.0,Home Movies,Home Movies,A cult guru urges a shy disciple to make life ...,1980-05-16,https://images-na.ssl-images-amazon.com/images...,89134.0,brian de palma,united states,us,en,english,400000.0,https://m.media-amazon.com/images/M/MV5BNWY4ZD...,NaN
3,80339.0,88,comedy,robert hays|julie hagerty|leslie nielsen,7.7,231364.0,Airplane!,Airplane!,"Alcoholic pilot, Ted Striker has developed a f...",1980-02-07,https://images-na.ssl-images-amazon.com/images...,83453539.0,jim abrahams|david zucker|jerry zucker,united states,us,en,english,3500000.0,https://m.media-amazon.com/images/M/MV5BZjA3Yj...,paramount pictures|howard w. koch productions
4,80360.0,102,horror|sci-fi|thriller,william hurt|blair brown|bob balaban,6.9,34558.0,Altered States,Altered States,A research scientist (William Hurt) explores t...,1980-12-25,https://images-na.ssl-images-amazon.com/images...,19853892.0,ken russell,united states,us,en|es,english|spanish,NaN,https://m.media-amazon.com/images/M/MV5BZTdkOT...,warner bros.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9259,1869514.0,115,comedy|music,guido kangur|roman baskin|harri kõrvits,5.4,282.0,Farts of Fury,Kormoranid ehk Nahkpükse ei pesta,"Kaiser, the lead singer of a former rock group...",2011-03-17,NaN,204395.0,andres maimik|rain tolk,estonia,ee,et|en,estonian|english,NaN,https://m.media-amazon.com/images/M/MV5BYTcyNz...,kuukulgur film
9265,1874412.0,90,adventure|animation|sci-fi,adam behr|bruce boxleitner|steve breen,4.5,246.0,AniMen - Triton Force,AniMen - Triton Force,"Over time, the people residing on Swampland be...",2010-03-24,NaN,395220.0,xu kerr,china|united states,cn|us,NaN,NaN,NaN,https://m.media-amazon.com/images/M/MV5BMjEyMT...,gorilla pictures
9388,1964479.0,120,documentary,NaN,6.8,65.0,American Passages,American Passages,"Initially, there's that moment of happiness: a...",2011-03-31,NaN,57450.0,ruth beckermann,austria,at,en,english,NaN,https://m.media-amazon.com/images/M/MV5BYmE1Y2...,ruth beckermann filmproduktion
9538,2061665.0,103,drama,david rees|darren kelfkens|leandie du randt,3.5,11.0,Egoli: The Movie,Egoli: The Movie,Niek and Joe share a history and a secret from...,2010-06-16,NaN,53736.0,bromley cawood,south africa,za,en|af,english|afrikaans,NaN,https://m.media-amazon.com/images/M/MV5BYzIyMD...,brigadiers franz marx films


In [8]:
max([len(i) for i in train_encodings['input_ids']])

296

In [10]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                                         problem_type='regression',
                                                         num_labels=1
                                                        )

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [67]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  learning_rate = 5e-05,
                                  weight_decay = 0.0,
                                  num_train_epochs = 3)

def get_model():
    return RobertaForSequenceClassification.from_pretrained("roberta-base",
                                                         problem_type='regression',
                                                         num_labels=1
                                                        )

trainer = Trainer(
    model_init=get_model(),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /Users/samirchar/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 

RuntimeError: model_init should have 0 or 1 argument.

In [27]:
18750->500


roberta.embeddings.word_embeddings.weight True
roberta.embeddings.position_embeddings.weight True
roberta.embeddings.token_type_embeddings.weight True
roberta.embeddings.LayerNorm.weight True
roberta.embeddings.LayerNorm.bias True
roberta.encoder.layer.0.attention.self.query.weight True
roberta.encoder.layer.0.attention.self.query.bias True
roberta.encoder.layer.0.attention.self.key.weight True
roberta.encoder.layer.0.attention.self.key.bias True
roberta.encoder.layer.0.attention.self.value.weight True
roberta.encoder.layer.0.attention.self.value.bias True
roberta.encoder.layer.0.attention.output.dense.weight True
roberta.encoder.layer.0.attention.output.dense.bias True
roberta.encoder.layer.0.attention.output.LayerNorm.weight True
roberta.encoder.layer.0.attention.output.LayerNorm.bias True
roberta.encoder.layer.0.intermediate.dense.weight True
roberta.encoder.layer.0.intermediate.dense.bias True
roberta.encoder.layer.0.output.dense.weight True
roberta.encoder.layer.0.output.dense.bia

In [30]:
18750/500

37.5

In [29]:
1533*500/18750

40.88

In [31]:
1533/41

37.390243902439025

In [26]:
next(model.named_parameters())[1]

Parameter containing:
tensor([[ 0.1476, -0.0365,  0.0753,  ..., -0.0023,  0.0172, -0.0016],
        [ 0.0156,  0.0076, -0.0118,  ..., -0.0022,  0.0081, -0.0156],
        [-0.0347, -0.0873, -0.0180,  ...,  0.1174, -0.0098, -0.0355],
        ...,
        [ 0.0304,  0.0504, -0.0307,  ...,  0.0377,  0.0096,  0.0084],
        [ 0.0623, -0.0596,  0.0307,  ..., -0.0920,  0.1080, -0.0183],
        [ 0.1259, -0.0145,  0.0332,  ...,  0.0121,  0.0342,  0.0168]],
       requires_grad=True)

In [10]:
trainer.train()

/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8177
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3069


Step,Training Loss


KeyboardInterrupt: 

In [58]:
def foo(c,a,b):
    print(c,a,b)
    return a+b

In [64]:
foo(123,**{'a':1,'b':22})

123 1 22


23

In [53]:
f"**{'a':10,'b':22}"

ValueError: Invalid format specifier

In [112]:

gammas = [0.95,0.99]
lrs = [0.05,0.1,0.2]
eps_start = np.arange(.7,1.1,.1).tolist()
eps_end = [0.001,0.01,0.1]
decays = [.99,.999,.9999]
repeats = 5

hparams = [lrs,eps_start,eps_end,gammas,decays]

combs = list(product(*[range(len(i)) for i in hparams],range(repeats)))
scores = np.zeros((*[len(i) for i in hparams],repeats))

In [113]:
hparams

[[0.05, 0.1, 0.2],
 [0.7,
  0.7999999999999999,
  0.8999999999999999,
  0.9999999999999999,
  1.0999999999999999],
 [0.001, 0.01, 0.1],
 [0.95, 0.99],
 [0.99, 0.999, 0.9999]]

In [100]:
combs

[(0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 2),
 (0, 0, 0, 0, 0, 3),
 (0, 0, 0, 0, 0, 4),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 1, 2),
 (0, 0, 0, 0, 1, 3),
 (0, 0, 0, 0, 1, 4),
 (0, 0, 0, 0, 2, 0),
 (0, 0, 0, 0, 2, 1),
 (0, 0, 0, 0, 2, 2),
 (0, 0, 0, 0, 2, 3),
 (0, 0, 0, 0, 2, 4),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1),
 (0, 0, 0, 1, 0, 2),
 (0, 0, 0, 1, 0, 3),
 (0, 0, 0, 1, 0, 4),
 (0, 0, 0, 1, 1, 0),
 (0, 0, 0, 1, 1, 1),
 (0, 0, 0, 1, 1, 2),
 (0, 0, 0, 1, 1, 3),
 (0, 0, 0, 1, 1, 4),
 (0, 0, 0, 1, 2, 0),
 (0, 0, 0, 1, 2, 1),
 (0, 0, 0, 1, 2, 2),
 (0, 0, 0, 1, 2, 3),
 (0, 0, 0, 1, 2, 4),
 (0, 0, 1, 0, 0, 0),
 (0, 0, 1, 0, 0, 1),
 (0, 0, 1, 0, 0, 2),
 (0, 0, 1, 0, 0, 3),
 (0, 0, 1, 0, 0, 4),
 (0, 0, 1, 0, 1, 0),
 (0, 0, 1, 0, 1, 1),
 (0, 0, 1, 0, 1, 2),
 (0, 0, 1, 0, 1, 3),
 (0, 0, 1, 0, 1, 4),
 (0, 0, 1, 0, 2, 0),
 (0, 0, 1, 0, 2, 1),
 (0, 0, 1, 0, 2, 2),
 (0, 0, 1, 0, 2, 3),
 (0, 0, 1, 0, 2, 4),
 (0, 0, 1, 1, 0, 0),
 (0, 0, 1, 1, 0, 1),
 (0, 0, 1, 1,

In [135]:
hparams = {'gamma' : [0.95,0.99],
           'lr' : [0.05,0.1,0.2],
           'eps_start' : np.arange(.7,1.1,.1).tolist(),
           'eps_end' : [0.001,0.01,0.1],
           'decay' : [.99,.999,.9999],
           'repeats': range(5)}

combs = list(product(*[range(len(i)) for i in list(hparams.values())]))
scores = np.zeros([len(i) for i in list(hparams.values())])


for comb_indexes in combs:
    comb_values = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    score = 1
    scores[tuple(comb_indexes)] = score

In [230]:
comb_values

{'gamma': 0.99,
 'lr': 0.2,
 'eps_start': 1.0999999999999999,
 'eps_end': 0.1,
 'decay': 0.9999,
 'repeats': 4}

'0.95 0.05 0.7 0.001 0.99 0.0'

array(['0.95', '0.05', '0.7', '0.001', '0.99', '0.0'], dtype='<U32')

In [172]:
scores

array([[[[[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]],

          [[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]],


         [[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]],

          [[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]],


         [[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]],

          [[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]]],



        [[[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]],

          [[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]]],


         [[[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.]],

          [[1., 1., 1., 1., 1.],
           [1., 1., 1., 1., 1.],
           [1.,

In [ ]:
6250->500


In [234]:
(11000//16)*3

2061

In [59]:
tokenizer

PreTrainedTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})